# teste json astronomia

In [ ]:
!pip install chatterbot chatterbot_corpus spacy==3.7.4 -q
!python -m spacy download pt_core_news_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 645.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.5/240.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/

--------------------

In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import json
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.languages import POR

In [ ]:
nome_arquivo_json = 'perguntas_astronomia_langchain.json'
conversa_para_treino = []

try:
    with open(nome_arquivo_json, 'r', encoding='utf-8') as f:
        dados_json = json.load(f)

    for item in dados_json:
        if 'pergunta' in item and 'resposta' in item:
            conversa_para_treino.append(item['pergunta'])
            conversa_para_treino.append(item['resposta'])
        else:
            print(f"Aviso: Item inválido no JSON (sem 'pergunta' ou 'resposta'): {item}")

except FileNotFoundError:
    print(f"Erro: Arquivo '{nome_arquivo_json}' não encontrado.")
    print("Certifique-se de que o arquivo JSON está no mesmo diretório que o script.")
    exit()
except json.JSONDecodeError:
    print(f"Erro: O arquivo '{nome_arquivo_json}' não contém um JSON válido.")
    exit()
except Exception as e:
    print(f"Ocorreu um erro inesperado ao ler o JSON: {e}")
    exit()

if not conversa_para_treino:
    print("Erro: Nenhum dado válido encontrado no JSON para treinamento.")
    exit()

In [ ]:
chatbot_astro = ChatBot(
    "BotAstronomico",
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    database_uri='sqlite:///db_astronomia.sqlite3',
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch',
            'default_response': 'Desculpe, não entendi bem. Poderia reformular?',
            'maximum_similarity_threshold': 0.9
        },
    ],
    tagger_language=POR
)

trainer_astro = ListTrainer(chatbot_astro) #json file trainer
print(f"Iniciando treinamento com {len(conversa_para_treino) // 2} pares de pergunta/resposta...")
trainer_astro.train(conversa_para_treino)
print("Treinamento concluído!")

Iniciando treinamento com 541 pares de pergunta/resposta...
List Trainer: [####################] 100%
Treinamento concluído!


In [ ]:
print("\nOlá! Sou o Bot Astronômico. Pergunte-me algo sobre astronomia (digite 'parar' para sair).")
while True:
    try:
        mensagem = input("Você: ")
        if mensagem.lower() == "parar":
            print("Bot: Até a próxima!")
            break
        resposta = chatbot_astro.get_response(mensagem, read_only=True)
        print(f"Bot: {resposta}")
    except (KeyboardInterrupt, EOFError):
        print("\nBot: Encerrando...")
        break
    except Exception as e:
        print(f"Ocorreu um erro inesperado durante a conversa: {e}")
        break




Olá! Sou o Bot Astronômico. Pergunte-me algo sobre astronomia (digite 'parar' para sair).
Você: o que é estrela?
Bot: Desculpe, não entendi bem. Poderia reformular?
Você: Sol
Bot: Em cerca de 5 bilhões de anos, o Sol se expandirá para uma gigante vermelha, engolindo os planetas internos. Depois, ejetará suas camadas externas (formando uma nebulosa planetária) e seu núcleo se tornará uma anã branca, que esfriará lentamente.
Você: Júpiter 
Bot: Atualmente, Júpiter tem 95 luas confirmadas. Esse número pode mudar à medida que novas luas são descobertas. As quatro maiores luas, conhecidas como luas galileanas (Io, Europa, Ganímedes e Calisto), foram descobertas por Galileu Galilei em 1610.
Você: Saturno 
Bot: Os anéis de Saturno são compostos principalmente por incontáveis partículas de gelo de água, com uma quantidade menor de poeira e material rochoso. Essas partículas variam em tamanho, desde grãos microscópicos até pedaços do tamanho de montanhas. Embora pareçam sólidos vistos da Terra

In [ ]:
"""from chatterbot.trainers import ChatterBotCorpusTrainer
trainer_astro = ChatterBotCorpusTrainer(chatbot_astro)
trainer_astro.train("/content/perguntas_astronomia_langchain.json")"""

AttributeError: 'list' object has no attribute 'get'